In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# データの読み込み
train_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/train.csv',nrows = 1000000)
test_df = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/test.csv')


In [ ]:
# テストデータの確認
train_df


In [ ]:
# テストデータの確認
test_df


In [ ]:
# 訓練データの要素（列名）を確認
train_df.columns


In [ ]:
# テストデータの要素（列名）を確認
test_df.columns


In [ ]:
# 訓練データの列ごとの欠損値の個数を確認
train_df.isnull().sum().sort_values(ascending = False)


In [ ]:
# テストデータの列ごとの欠損値の個数を確認
test_df.isnull().sum().sort_values(ascending = False)


In [ ]:
# 訓練データの欠損値ある行を確認
a = train_df[train_df["dropoff_longitude"].isnull()|train_df["dropoff_latitude"].isnull()]
print(a)


In [ ]:
# 訓練データの欠損値ある行を削除
train_df.drop(a.index,axis = 0, inplace = True)


In [ ]:
# 訓練データの欠損値が削除されていることを確認
train_df.isnull().sum().sort_values(ascending = False)


In [ ]:
# 訓練データの統計量を確認
train_df.describe()


In [ ]:
# テストデータの統計量を確認
test_df.describe()


In [ ]:
# ヒストグラム（料金）
sns.histplot(
data = train_df, x = 'fare_amount', kde = True, bins = 100)


In [ ]:
# 箱ひげ図（料金）
sns.boxplot(
data = train_df, y = 'fare_amount')


In [ ]:
train_df['passenger_count'].describe()


In [ ]:
# ヒストグラム（乗客人数）
sns.histplot(
data = train_df, x = 'passenger_count')
plt.xlim(0,10)
plt.ylim(0,1000000)


In [ ]:
# 料金の範囲を2.5～100ドルで指定
train_df = train_df[(train_df['fare_amount'] >= 2.5) & (train_df['fare_amount'] < 100)]


In [ ]:
train_df['fare_amount'].describe()


In [ ]:
# ヒストグラム（料金）変化を確認
sns.histplot(
data = train_df, x = 'fare_amount', kde = True, bins = 100 )
plt.xlim(0,80)
plt.ylim(0,150000)


In [ ]:
sns.boxplot(
data = train_df, y = 'fare_amount')


In [ ]:
# 乗客人数の範囲を0～6人で指定・ヒストグラム（乗客人数）の確認
train_df = train_df[(train_df['passenger_count'] <= 6) & (train_df['passenger_count'] >= 1)]
sns.histplot(
data = train_df, x = 'passenger_count')
plt.ylim(0,1000000)


In [ ]:
## 経度：42~40.5,緯度：-74.5~-73の範囲で指定
train_df = train_df[(train_df['pickup_longitude'] <= -73.0) & (train_df['pickup_longitude'] >= -74.5)]
train_df = train_df[(train_df['pickup_latitude'] >= 40.5) & (train_df['pickup_latitude'] <= 42)]
train_df = train_df[(train_df['dropoff_longitude'] <= -73.0) & (train_df['dropoff_longitude'] >= -74.5)]
train_df = train_df[(train_df['dropoff_latitude'] >= 40.5) & (train_df['dropoff_latitude'] <= 42)]


In [ ]:
train_df.describe()


In [ ]:
#これまでの処理によるデータを確認
train_df


In [ ]:
train_df.describe()


In [ ]:
#散布図（乗客人数 vs. 料金）
sns.scatterplot(data = train_df, x = 'passenger_count', y = 'fare_amount')


In [ ]:
# 料金予測に使用できない要素を削除
train_df.drop(['key', 'pickup_datetime'], axis = 1, inplace = True)
test_df.drop(['key', 'pickup_datetime'], axis = 1, inplace = True)


In [ ]:
train_df.columns


In [ ]:
test_df.columns


In [ ]:
# 経緯度の情報では予測に使用できないため、経緯度から距離を算出する関数を設定
from geopy.distance import geodesic
def get_distance_km(lat1, lon1, lat2, lon2):
    p_point = (lat1, lon1)
    d_point = (lat2, lon2)
    distance_km = geodesic(p_point, d_point).km
    return distance_km


In [ ]:
# 訓練データの行数
train_len = len(train_df)
print(train_len)


In [ ]:
# 訓練データとテストデータを結合
df = pd.concat([train_df,test_df], axis = 0)


In [ ]:
print(len(df))


In [ ]:
# 作成した関数から距離を算出
distances_km = []
for i in range(len(df)):
    distance = get_distance_km(df.iloc[i,df.columns.get_loc('pickup_latitude')],df.iloc[i,df.columns.get_loc('pickup_longitude')],df.iloc[i,df.columns.get_loc('dropoff_latitude')],df.iloc[i,df.columns.get_loc('dropoff_longitude')])
    distances_km.append(distance)


In [ ]:
# distances_kmのlistを20個確認
print(distances_km[:20])


In [ ]:
# データに距離の要素を追加
df['distance'] = distances_km


In [ ]:
df


In [ ]:
# 結合したデータを訓練データとテストデータに分割
train = df[0:train_len]
test = df[train_len:]


In [ ]:
train.isnull().sum().sort_values(ascending = False)


In [ ]:
test.isnull().sum().sort_values(ascending = False)


In [ ]:
# 距離を確認
train['distance'].describe()


In [ ]:
test['distance'].describe()


In [ ]:
# 箱ひげ図（距離）
sns.boxplot(
data = train, y = 'distance')


In [ ]:
# 外れ値を削除
train = train[train['distance'] < 40]


In [ ]:
#散布図（料金 vs. 距離）
sns.scatterplot(data = train, x = 'distance', y = 'fare_amount')


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


In [ ]:
# 料金とそれ以外の特徴量に分割
X = train.drop(['fare_amount'],axis = 1)
y = train['fare_amount']


In [ ]:
# トレーニングセットとテストセットに分割
train_X, valid_X, train_y, valid_y = train_test_split(X,y,test_size = 0.2,random_state = 42)


In [ ]:
print(len(valid_y))


In [ ]:
# XGBoost用に変換/訓練データと評価用データを設定
dtrain = xgb.DMatrix(train_X, label = train_y)
dvalid = xgb.DMatrix(valid_X, label = valid_y)


In [ ]:
# ハイパーパラメータの設定
params = {
    'objective': 'reg:squarederror',  # 回帰問題の場合は 'reg:squarederror'
    'eval_metric': 'rmse',            # RMSEで評価
    'max_depth': 5,                   # ツリーの深さ
    'learning_rate': 0.1,             # 学習率
    'subsample': 0.8,                 # サンプリング率
    'colsample_bytree': 0.8,          # ツリーを構築する際に考慮する特徴量の割合
    'verbosity': 0,                   # 処理の詳細度（0で非表示）
    'seed': 42                        # 乱数シード
}


In [ ]:
# モデルのトレーニング
num_round = 100
model = xgb.train(params, dtrain, num_round)


In [ ]:
# 評価用データを用いた評価・平均二乗誤差（回帰モデル）
y_pred = model.predict(dvalid)
mse = mean_squared_error(valid_y, y_pred)
print("Mean Squared Error:", mse)


In [ ]:
test


In [ ]:
# テストデータの分割
test_X = test.drop(['fare_amount'],axis = 1)
test_y = test['fare_amount']


In [ ]:
# 予測
dtest = xgb.DMatrix(test_X)
y_pred = model.predict(dtest)


In [ ]:
# 提出用データを読み込む
sub = pd.read_csv('/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv')


In [ ]:
# 提出ファイルの作成
submission = pd.DataFrame({'key': sub['key'], 'fare_amount': y_pred})
submission.to_csv('submission.csv', index=False)


In [ ]:
# 確認
submission
